In [1]:
from wallet_database_sqlite3 import Sqlite3Wallet

In [2]:
db = Sqlite3Wallet()

connection with database made.


In [3]:
db.conn

In [4]:
#db.erase_database()

database is locked
database is locked


True

In [3]:
#res = db.create_tables()
#res
db.create_CorporateAccount_table()

CREATE TABLE IF NOT EXISTS CorporateAccount ( acc_index INT NOT NULL, account_name text NOT NULL, 
        wallet_name text NOT NULL,
PRIMARY KEY (acc_index, wallet_name) ) WITHOUT ROWID;


True

In [6]:
res = db.does_table_exist("CorporateAccount")
res

[('CorporateAccount',)]

In [9]:
#f = db.cursor().execute("DROP TABLE Addresses")
#db.create_contact_table()
contact_list_raw = db.get_all_contacts()
contact_list_raw
contact_list  = [x[1]+" "+x[0] for x in contact_list_raw if x[1].startswith("o") or x[0].startswith("")]
contact_list

SELECT first_name, last_name, phone, position, xpub FROM Contacts



['perez pepito']

In [8]:
#db.close()

In [3]:
xprv ="tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP"
words = 'b"engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"'[1:]
res = db.new_wallet(xprv, words, "from jupyter notebook")

INSERT INTO Wallets (xprv, words, name)
 VALUES('tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP', "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning", 'from jupyter notebook') ;


In [4]:
res

True

In [5]:
res = db.new_address("first_address", "m/0H/0H/", 0, 0, xprv)
res

True

In [12]:
res = db.new_utxo("first_address", 22222, "transaction_0", 0, confirmed=6)
res

True

In [13]:
utxos = db.look_for_coins(xprv)

In [14]:
utxos

[('transaction_0', 0, 22222)]

In [15]:
unused_addresses = db.get_unused_addresses(xprv)
unused_addresses

[]

In [6]:
res = db.new_address("second_address", "m/0H/0H/", 1, 0, xprv)
res

True

In [11]:
res = db.new_address("Thirteenth_address", "m/0H/0H/", 13, 0, xprv)
res

True

In [17]:
unused_addresses = db.get_unused_addresses(xprv, 2,2)
unused_addresses

SELECT address 
FROM Addresses WHERE
NOT EXISTS(
SELECT 1 
 FROM Utxos
WHERE Utxos.address = Addresses.address) 
AND
 Addresses.wallet = 'tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP'
AND
 (1591753283 - Addresses.created) > 0 AND (1591753283 - Addresses.created) < 172800 ;


[('second_address',)]

In [18]:
db.get_all_addresses(xprv)

[('first_address',), ('second_address',)]

In [3]:
f = db.conn.cursor().execute("SELECT * FROM CorporateAccount")

In [4]:
f.fetchall()
#f.description

[(2, 'DEVELOPERS', 'My wallet #2')]

In [20]:
db.new_tx("tx_n1",[("transaction_0",0)],[(1000,"script_public_key of my first transaction")])

True

In [21]:
g = db.conn.cursor().execute("SELECT * FROM Transactions")
g.fetchall()

[('tx_n1', 0, 1, 0, 1591752025)]

In [22]:
h = db.conn.cursor().execute("SELECT * FROM Tx_Ins")
h.fetchall()

[('transaction_0', 0, 'tx_n1')]

In [3]:
i = db.conn.cursor().execute("SELECT * FROM Tx_Outs")
i.fetchall()

[(0,
  1000,
  '00508e440dc876e769e0dd3e2f61f8e7d33ce90a2e8f023c9aa9f0b8e48c4888',
  'This is a test ScriptPubKey')]

In [24]:
db.conn.cursor().execute("DROP TABLE Wallets")

In [25]:
db.does_table_exist("Addresses")

[('Addresses',)]

In [4]:
db.close_database()

In [28]:
account = "m/0H/1H/"
g = db.conn.cursor().execute(f"SELECT MAX(acc_index) FROM Addresses WHERE wallet='{xprv}' AND path='{account}';")
j = g.fetchall()[0][0]
j

In [29]:
if j is None:
    print("yes")

yes


In [8]:
wallet = "tprv8ZgxMBicQKsPcv5EhtDXAeg624gwoQpnB55T1qaUPbgzjUrziZ9VkM2es6MsMBe7tRpvnoGWb9RFoKjhpkovSepJiMFX5NWBVtcpWcd96pp"
query1 = f"SELECT tx_id, out_index, amount\n FROM Utxos INNER JOIN Addresses \n"
#query1 = f"SELECT tx_id, out_index, amount\n FROM Utxos;"
query2 = f"ON Utxos.address = Addresses.address\nWHERE Utxos.spent = 0 AND Addresses.wallet = '{wallet}';"
#query2 = f"ON Utxos.address = Addresses.address\nWHERE Utxos.spent = 1;"
query = query1 + query2
i = db.conn.cursor().execute(query)
i.fetchall()

[('ce8989c24a699924ecf8281b0b88f1f04de9a98ada0d24fd24f053a2ece21b5e',
  1,
  1000000),
 ('71ae64055a84d489aa9f92bb53ad1d76dd846cbafc614a16893c1adf5023c6eb',
  1,
  22000)]

In [13]:
query= "UPDATE Utxos SET spent = 1 WHERE tx_id = 'fb879418a9f754cc4d050c37fcb2af4e0b8e65a7abc7213828f0d507cd1c24e2' AND out_index = 1"
i = db.conn.cursor().execute(query)

In [14]:
query= "SELECT spent FROM Utxos WHERE tx_id = 'fb879418a9f754cc4d050c37fcb2af4e0b8e65a7abc7213828f0d507cd1c24e2' AND out_index = 1"
i = db.conn.cursor().execute(query)
i.fetchall()

[(1,)]

In [15]:
db.conn.commit()


In [10]:
from wallet_database_sqlite3 import Sqlite3Environment


env = Sqlite3Environment()
env

connection with database made.


In [11]:
env.new_key("BLOCKCYPHER_API_KEY","d3e470c810b6440ca60e57f1f60939c6")
env.new_key("CC_API","e57bbc8169583b0ebfba6e517836f199eab4ffa74a752d54fcfd2cd0cc385641")

INSERT INTO Keys (name, key)
VALUES('BLOCKCYPHER_API_KEY', 'd3e470c810b6440ca60e57f1f60939c6') ;
INSERT INTO Keys (name, key)
VALUES('CC_API', 'e57bbc8169583b0ebfba6e517836f199eab4ffa74a752d54fcfd2cd0cc385641') ;


True

In [12]:
i = env.conn.cursor().execute("SELECT * FROM Keys")
i.fetchall()

[('BLOCKCYPHER_API_KEY', 'd3e470c810b6440ca60e57f1f60939c6'),
 ('CC_API',
  'e57bbc8169583b0ebfba6e517836f199eab4ffa74a752d54fcfd2cd0cc385641')]

In [19]:
query= "SELECT * FROM SHDSafeWallet"
i = db.conn.cursor().execute(query)
i.fetchall()

[('',
  '[242181816824935352779949905339585673702570459364503992825384375679130606102141, 243939266812165980563239760114596239186104406092581261489726331066308569928747, 290372254706026096153481489592931453085780088683353495329679236536122868177482, 376644786442849392304182314585976636972636060283971456632342867625859504056500, 461222389963770672910023206337944803706847222730930779040024025202330161562314]',
  2,
  6,
  'None',
  'tprv8ghfWd73PvMPqRazzw1m6Vxq6TUzVmbuiyfQ4frvAXgM8kPidJ7QWDWPRmwF9J3da2EhRnLJjiwwFKYAo4bW8p82JRntd8MtKydwTpDoiGj',
  'tpubDDPhf39HYJ34itcntagMVucwfUzvf6npJHGBMBuDaoUjyEeVFgvzgi8FbuQnRdMzMDd81kiqs7zcsGQRFC3B75QQQbVbtUM5cqnr7gWuqdf',
  'p2wsh',
  1,
  1,
  'None',
  -1),
 ('200823',
  '[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 34904519403285236524485843581086005758991025139939414745136436266219764179988]',
  2,
  4,
  'None',
  'tprv8oQnacENnx7r

In [13]:
query= "SELECT tx_id,out_index FROM Tx_Ins WHERE spent_by = 'de14fbd1f89df79e4de29df6890af3db9d7c4511173e4f9e70f100d6cb17655a'"
i = db.conn.cursor().execute(query)
i.fetchall()

[('5324623d891ff225b41570d09320770c30d9291a0193cda09504a115fb661d1c', 0)]

In [15]:
query1 = "SELECT spent FROM Utxos WHERE tx_id = ("
query2 = "SELECT tx_id FROM Tx_Ins WHERE spent_by = 'de14fbd1f89df79e4de29df6890af3db9d7c4511173e4f9e70f100d6cb17655a') "
query3 = "AND out_index = (SELECT out_index FROM Tx_Ins WHERE spent_by = 'de14fbd1f89df79e4de29df6890af3db9d7c4511173e4f9e70f100d6cb17655a')"

query = query1 + query2 + query3
i = db.conn.cursor().execute(query)
i.fetchall()

[(1,)]

In [ ]:
query1 = "UPDATE Utxos SET spent = 0 WHERE ("
query2 = "SELECT tx_id,out_index FROM Tx_Ins WHERE spent_by = 'de14fbd1f89df79e4de29df6890af3db9d7c4511173e4f9e70f100d6cb17655a')"
query = query1 + query2
i = db.conn.cursor().execute(query)
i.fetchall()

In [14]:
query = "ALTER TABLE Contacts ADD COLUMN safe_pubkey text"
w = db.conn.cursor().execute(query)

In [15]:
db.conn.commit()


In [3]:
query = "SELECT * \nFROM addresses"
w = db.conn.cursor().execute(query)
w.description

(('address', None, None, None, None, None, None),
 ('acc_index', None, None, None, None, None, None),
 ('path', None, None, None, None, None, None),
 ('change_addr', None, None, None, None, None, None),
 ('created', None, None, None, None, None, None),
 ('wallet', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None),
 ('safe_index', None, None, None, None, None, None))

In [20]:
query = "ALTER TABLE SHDSafeWallet ADD COLUMN level1pubkeys text"
w = db.conn.cursor().execute(query)
db.conn.commit()

In [11]:
query = "SELECT * FROM wallets WHERE 1=0"
w = db.conn.cursor().execute(query)
w.description

(('xprv', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('words', None, None, None, None, None, None),
 ('child', None, None, None, None, None, None))

In [3]:
w = db.create_SHDSafeWallet_table()

In [4]:
db.conn.commit()

In [25]:
query1 = f"SELECT * FROM Partial_Tx_Ins\n"
query2 = f"WHERE spent_by ='01adcbc8a57d1fca226fd2502b784350670bd7345ba8cd385da7f3f6b53d99c0' ;"
w = db.conn.cursor().execute(query1 + query2)
y = w.fetchall()

In [27]:
tx_ins = [(x[0],x[1]) for x in y]
tx_ins

[('e43843e9eff42b4a2a74241bf44a959d6283963110d20dc83fbd224a408b8734', 1)]

In [20]:
tx_id = "ffac4dd0cf580305db11dd206efb003d3337d4b7939d66168161f7851e0161d9"
query = f"SELECT * FROM Utxos WHERE tx_id = (SELECT tx_id FROM Tx_Ins WHERE spent_by = '{tx_id}') AND out_index = (SELECT out_index FROM Tx_Ins WHERE spent_by = '{tx_id}')"
#query = f"SELECT tx_id FROM Tx_Ins WHERE spent_by = '{tx_id}'"
#

w = db.conn.cursor().execute(query)
w.fetchall()

[('tb1q9w5elsaaslf8taqfrdlq846jsrkv5dlucqvt0r',
  85086,
  '5d54883400096d4c3db02cf9772c63e7fe1fd30415b73f8088991b52f2534cc7',
  1,
  1592351480,
  0,
  0)]

In [5]:
w.description

(('tx_id', None, None, None, None, None, None),
 ('lock_time', None, None, None, None, None, None),
 ('version', None, None, None, None, None, None),
 ('n_confirmations', None, None, None, None, None, None),
 ('created', None, None, None, None, None, None))

In [4]:
w = db.create_HDMWallet_table()
db.conn.commit()

In [10]:
query1 = """INSERT INTO PartialTransactions (tx_id,created,n_confirmations,lock_time,version,cosigners_reply,tx_hex)
 VALUES('695e082bf8bafe95fc2e722edb44b4fcf8c3ba4ebad6c2ff40ec5431bce8e42b',1606266937,0,0,1,"{'242181816824935352779949905339585673702570459364503992825384375679130606102141': None, '243939266812165980563239760114596239186104406092581261489726331066308569928747': None, 'tpubDF5r1amiE2pq81PKssn6aJtUebfDvnKZuwge73PHYW9g9qd4KsunL8Br9pSyCQ3dmLRHLYgiwi1zcBktqs2zzV22GFuykK5A7VKDgxJUF6N': True}",'01000000000101f5d786509a717d9591c6e93a1d0a2b8ab72fe33b802aa7655babacb4cb434dc00100000000ffffffff02de000000000000002200205ae31f55a2d06e4810746f9f41f5f496ecee2965a085c8122ab57858e3afe0d28364010000000000220020ed28021523355e748429edbe509259cb944baa2b26cfc527d617e6c40ad7a1f104000047304402201ecc50aea87142ef1be7b660f0ca25ba900b14c951480e05e9c1eadcb53a8d2802203ebf37d88a9c42ed5e2fc4fc61b0587e56af544e962ab701be5d5bd78b0a62920169522102176e0cfd845d27ce83291f431dda33738fcf2096a6f3f7c4abdeb37008ede67d21021b50bb7089e5a251121c701903cd2bc64f61c71a23afcf2447a4ed8b59a1142b210234826fa196c5bb5369e3e4a0da8598e5f3f2e1da4b9b8bba697cfca85ca81fcf53ae00000000');"""
w = db.conn.cursor().execute(query1)
#w.fetchall()

In [4]:
db.create_partial_transaction_table()
db.create_partial_tx_in_table()
db.create_partial_tx_out_table

True

In [18]:
x = b"\x02\x1bP\xbbp\x89\xe5\xa2Q\x12\x1cp\x19\x03\xcd+\xc6Oa\xc7\x1a#\xaf\xcf$G\xa4\xed\x8bY\xa1\x14+"

In [19]:
int.from_bytes(x,"big")

243939266812165980563239760114596239186104406092581261489726331066308569928747

In [13]:
db.conn.commit()

In [12]:
db.delete_tx("0996ce043e38f8f21921596392aea0ed7a879639b5ba92622c100a3fa2fc8381")

True

In [3]:
db.recover_utxo("30346826e0b88ba78dd2bd37bde9dd084b1ccd37624973cf1a392c19b9973a01")

tx_ins: []


True

In [3]:
utxos = [('8084da65d7c8f53ec110e02e874caabc1c3823c673179ac8e5e3d1329be52541', 1)]
db.check_utxos_are_spendable(utxos)

True

In [8]:
query= "SELECT tx_id FROM PartialTransactions"
i = db.conn.cursor().execute(query)
partial_txs = i.fetchall()

In [11]:
partial_txs

[('30346826e0b88ba78dd2bd37bde9dd084b1ccd37624973cf1a392c19b9973a01',),
 ('49a2f89c1a9a350cf28d8511421fce8bee917a95c1f8625b7588d02961de650b',),
 ('8747ddc3e8469be4542c37d8751f16af385e8b55ba2d3fbb5addd21aa516da7d',),
 ('c607c08a7aac67ae930416b93cdb7c62062758276f4e051f823a4892cdbac9a9',)]

In [13]:
for tx in partial_txs:
    res = db.delete_partial_tx(tx[0])
    print(res)

True
True
True
True


In [3]:
query = f"SELECT * FROM Addresses"

w = db.conn.cursor().execute(query)
y = w.fetchall()

In [4]:
y

[('2MuprzwpsgbzTghKu4VRBT8iFHgyuzM9d31',
  3,
  'm/44H/0H/0H/',
  0,
  1592008589,
  'tprv8ZgxMBicQKsPcv5EhtDXAeg624gwoQpnB55T1qaUPbgzjUrziZ9VkM2es6MsMBe7tRpvnoGWb9RFoKjhpkovSepJiMFX5NWBVtcpWcd96pp',
  6,
  None),
 ('2NFmfYxGvJUamqKT9xWt4tZEg3hmERqAd42',
  2,
  'm/44H/0H/0H/',
  0,
  1592007852,
  'tprv8ZgxMBicQKsPcv5EhtDXAeg624gwoQpnB55T1qaUPbgzjUrziZ9VkM2es6MsMBe7tRpvnoGWb9RFoKjhpkovSepJiMFX5NWBVtcpWcd96pp',
  6,
  None),
 ('Multisignature account: tb1q0whr89k65adyn8wes9gk0qm6x976evjsu2kz35gptj2qfekd4tls3a0quq',
  1,
  'm/1/',
  1,
  1598552221,
  '200827_test1_master_wallet',
  5,
  200827),
 ('miLX6JQUFpEayoZMmNN4RWTTZfSX8fdfu1',
  6,
  'm/44H/0H/0H/',
  0,
  1592420277,
  'tprv8ZgxMBicQKsPcv5EhtDXAeg624gwoQpnB55T1qaUPbgzjUrziZ9VkM2es6MsMBe7tRpvnoGWb9RFoKjhpkovSepJiMFX5NWBVtcpWcd96pp',
  2,
  None),
 ('mjem1LPGbA4ERqxrXai4dbsBSuxYdY4d3g',
  5,
  'm/44H/0H/0H/',
  0,
  1592023131,
  'tprv8ZgxMBicQKsPcv5EhtDXAeg624gwoQpnB55T1qaUPbgzjUrziZ9VkM2es6MsMBe7tRpvnoGWb9RFoKjhpkovSepJiMFX5NWB

In [4]:
query1 = "UPDATE Utxos SET spent = 1 WHERE "
query2 = "\ntx_id = 'de14fbd1f89df79e4de29df6890af3db9d7c4511173e4f9e70f100d6cb17655a'  AND out_index = 1"
query = query1 + query2
i = db.conn.cursor().execute(query)
i.fetchall()

[]